In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import *
from sklearn import metrics
from pymatreader import read_mat
import numpy as np

In [7]:
data = np.zeros(shape=(10,125,30000,20)) #song x chan x T x person
for i in range(10): # channel x T x participant = 125 x 30,000 x 20
    data[i] = read_mat('song{}_Imputed.mat'.format(i + 21))['data{}'.format(i + 21)][:,:30000,:]

In [8]:
data[0].shape

(125, 30000, 20)

In [9]:
data_chanavg = np.mean(data, axis=1) #shape = (10, 30K, 20)

In [10]:
ratings = read_mat('behavioralRatings.mat')['behavioralRatings'][:,:,1].T

In [11]:
ratings.shape

(10, 20)

In [13]:
pip install einops

Note: you may need to restart the kernel to use updated packages.


In [14]:
from einops import rearrange

In [15]:
dataflat = rearrange(data_chanavg, 's t u -> (s u) t') # song time person -> song+person time
ratingsflat = ratings.reshape(200)

In [18]:
dataflat.shape #person+song x time

(200, 30000)

In [20]:
clf = RandomForestRegressor()
x_train, x_test, y_train, y_test = train_test_split(dataflat, ratingsflat, test_size = 0.70, random_state = 5)
clf.fit(x_train, y_train)
y_pred_test = clf.predict(x_test)
diffs = []
for i in range(50):
    diffs.append( y_pred_test[i] - y_test[i] )
print(np.mean([x**2 for x in diffs])) #could be worse..

4.3432580000000005


In [21]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id='xxx',
                                                     client_secret='xxx')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [33]:
def getSpotifyFeatures(name):
    search = sp.search(q=name, type='track', limit=1)
    search_uri = search['tracks']['items'][0]['uri']
    search_features = sp.audio_features(search_uri)[0]
    search_songname = search['tracks']['items'][0]['name']
    search_artist = search['tracks']['items'][0]['artists'][0]['name']
    print(f'Found {search_songname} by {search_artist}')
    ret = [search_features['danceability'], search_features['energy'], search_features['key'], search_features['loudness'], \
           search_features['mode'], search_features['speechiness'], search_features['acousticness'], \
           search_features['instrumentalness'], search_features['liveness'], search_features['valence'], \
           search_features['tempo']]
    return ret

In [34]:
getSpotifyFeatures('First Fires')

Found First Fires by Bonobo
[0.606, 0.408, 8, -10.539, 1, 0.0362, 0.385, 0.29, 0.437, 0.211, 110.01]


[0.606, 0.408, 8, -10.539, 1, 0.0362, 0.385, 0.29, 0.437, 0.211, 110.01]

In [37]:
data_chanavgR = rearrange(data_chanavg, 's t u -> s u t')
data_chanavgR.shape

(10, 20, 30000)

In [47]:
songs = ['First Fires', 'Oino', 'Tiptoes Artist: Daedalus', 'Careless Love / Croquet Club', 'Lebanese Blonde', 'Canopee', 'Doing Yoga', \
         'Until the Sun Needs to Rise', 'Silent Shout', 'The Last Thing You Should Do']
feats = [getSpotifyFeatures(song) for song in songs] #10 x 11

Found First Fires by Bonobo
[0.606, 0.408, 8, -10.539, 1, 0.0362, 0.385, 0.29, 0.437, 0.211, 110.01]
Found Oino by LA Priest
[0.759, 0.723, 8, -6.615, 0, 0.0465, 0.263, 0.483, 0.0857, 0.964, 139.031]
Found Tiptoes by Daedelus
[0.444, 0.821, 5, -7.467, 0, 0.0501, 0.115, 0.0443, 0.109, 0.162, 148.003]
Found Careless Love by Croquet Club
[0.637, 0.259, 9, -14.924, 0, 0.0597, 0.984, 0.679, 0.0917, 0.597, 82.495]
Found Lebanese Blonde by Thievery Corporation
[0.808, 0.837, 9, -6.583, 0, 0.0459, 0.238, 0.862, 0.0578, 0.654, 92.002]
Found Canopée by Polo & Pan
[0.704, 0.692, 5, -6.558, 0, 0.0304, 0.0942, 0.0109, 0.0922, 0.369, 96.005]
Found Doing Yoga by Kazy Lambist
[0.732, 0.76, 4, -6.82, 0, 0.0922, 0.214, 0.00112, 0.389, 0.318, 108.993]
Found Until the Sun Needs to Rise by RÜFÜS DU SOL
[0.726, 0.691, 8, -8.033, 1, 0.0337, 0.011, 0.294, 0.0916, 0.118, 119.996]
Found Silent Shout by The Knife
[0.599, 0.709, 2, -10.565, 1, 0.0396, 0.0983, 0.911, 0.119, 0.387, 128.561]
Found The Last Thing You

In [54]:
dataWFeats = np.zeros((10,20,30011))
for i, song in enumerate(songs):
    #data_chanavg[i]: 30,000(time) x 20(ppl)
    for j in range(20):
        #print(data_chanavgR[i][j])
        #data_chanavgR[i][j].append(feats[i])
        dataWFeats[i][j] = np.append(data_chanavgR[i][j], feats[i])

In [55]:
dataWFeats.shape

(10, 20, 30011)

In [58]:
dataWFeatsFlat = rearrange(dataWFeats, 's t u -> (s t) u') #song person time -> song+person time
dataWFeatsFlat.shape

(200, 30011)

In [63]:
clf = RandomForestRegressor()
x_train, x_test, y_train, y_test = train_test_split(dataWFeatsFlat, ratingsflat, test_size = 0.70, random_state = 5)
clf.fit(x_train, y_train)
y_pred_test = clf.predict(x_test)
diffs = []
for i in range(50):
    diffs.append( y_pred_test[i] - y_test[i] )
print(np.mean([x**2 for x in diffs])) #could be worse..

4.159658


In [60]:
clf = AdaBoostRegressor()
clf.fit(x_train, y_train)
y_pred_test = clf.predict(x_test)
diffs = []
for i in range(50):
    diffs.append( y_pred_test[i] - y_test[i] )
print(np.mean([x**2 for x in diffs]))

4.2071583182409285


In [64]:
from sklearn import neighbors
clf = neighbors.KNeighborsRegressor()
clf.fit(x_train, y_train)
y_pred_test = clf.predict(x_test)
diffs = []
for i in range(50):
    diffs.append( y_pred_test[i] - y_test[i] )
print(np.mean([x**2 for x in diffs]))

4.128000000000001
